# SAM: TP1 Accès aux données avec index 

Sujet pour étudiants

date de modification : 26/01/2023 16h

NOM: Zeng

Prénom: Fanxiang

Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage 

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de Sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [1]:
import os
import shutil as sh
import numpy as np
import random
from random import choice
from string import ascii_lowercase
import time

DATA = "data.csv"

# Générer un fichier

Création du fichier

In [2]:
# dure environ 40s pour 5M lignes

nb_lines = 5 * 1000 * 1000
# nb_lines = 100
nb_attributes = 7

longueur_attribut = 100
# string_val = "".join(choice(ascii_lowercase) for i in range(longueur_attribut))
long_string = ''.join('-' for i in range(longueur_attribut))

# a=[np.random.randint(0, int(nb_lines/(10**i)), nb_lines) for i in range(nb_attributes)]
nb_valeurs_distinctes = nb_lines

# le premier attribut est unique
a = [random.sample(range(nb_valeurs_distinctes), nb_lines)]

# les attributs suivants ont des domaines plus petits
for i in range(1, nb_attributes):
  nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
  a.append(np.random.randint(0, nb_valeurs_distinctes, nb_lines))

b = [ ','.join(map(lambda x: str(x), e)) + f",{long_string}\n" for e in zip(*a)]

with open(DATA, "w") as f:
  f.write(''.join(b))

In [3]:
%%bash
echo "head : "
head -n 2 data.csv
echo "tail : "
tail -n 2 data.csv
echo "size (lines) :"
wc -l data.csv

head : 
668325,375775,777175,199793,91868,1263,53883,----------------------------------------------------------------------------------------------------
2883814,1775781,529105,365072,38246,126521,53449,----------------------------------------------------------------------------------------------------
tail : 
1934621,1808134,838894,20873,70264,150259,24485,----------------------------------------------------------------------------------------------------
4904763,1423918,61818,585894,74440,75926,30931,----------------------------------------------------------------------------------------------------
size (lines) :
 5000000 data.csv


# Lecture séquentielle

In [4]:
def filtrer_fichier(fichier, valeur_recherchee):
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      a = int(line.split(',')[0])
      if a == s :
        print(f"ligne {i} :", line.strip())


s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier(DATA, s)
print("done in", time.time() - t1, "s")

valeur recherchée : 12413
ligne 1719411 : 12413,1149164,862575,486975,172155,7809,53812,----------------------------------------------------------------------------------------------------
done in 2.0054519176483154 s


# Découper le fichier en pages

In [5]:
def page_dir_name(fichier):
  return fichier.split('.')[0] + "_pages"

def decoupe_fichier_en_pages(fichier, nb_tuple_par_page):
  page_dir = page_dir_name(fichier)
  print("pages dans :", page_dir)
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  with open(fichier, "r") as f:
    p=0
    lines = []
    for i, line in enumerate(f):
      lines.append(line)
      if (i+1) % nb_tuple_par_page == 0:
        p += 1
        with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
        lines = []
    if len(lines) > 0:
      p +=1
      with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
    
    print("nb pages créées :", p)

decoupe_fichier_en_pages(DATA, nb_tuple_par_page=1000)

pages dans : data_pages
nb pages créées : 5000


Afficher le nombre de tuples dans une page (pour quelques pages)

In [6]:
%%bash
wc -l data_pages/* | head -n 3

    1000 data_pages/page1
    1000 data_pages/page10
    1000 data_pages/page100


# Lecture séquentielle du fichier découpé en pages

In [7]:
def lecture_sequentielle_par_page(fichier):
    page_dir = page_dir_name(fichier)
    nb_pages = len(os.listdir(page_dir))
    for p in range(1, nb_pages+1):
        with open(page_dir + f"/page{p}", "r") as fp:
            for i, line in enumerate(fp):
                yield (p, i, line.strip().split(','))

#   # a faire : pour chaque page, lire ses lignes
#   # une ligne devient un tuple
#   # retourner un itérateur contenant le numéro de page, la position dans la page et le tuple


def filtrer_fichier_par_pages(fichier, valeur_recherchee):
    for p, i, t in lecture_sequentielle_par_page(fichier):
        if int(t[0]) == valeur_recherchee:
            print(f"ligne {i} de la page {p} :", t)

#   # a faire pour chaque (numéro de page, position dans la page, tuple) obtnenu en invoquan la méthode ci dessus
#   # convertir le 1er attribut en un nombre l'afficher si il est egal à la valeur recherchee



s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier_par_pages("data.csv", s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 43973
ligne 409 de la page 3959 : ['43973', '369671', '1054476', '9150', '124715', '82616', '65703', '----------------------------------------------------------------------------------------------------']
done in 3.12 s


# Lecture d'un tuple dans une page

In [8]:
def lecture_tuple(fichier, num_page, position):
    with open(page_dir_name(fichier) + f"/page{num_page}", "r") as fp:
        for i, line in enumerate(fp):
            if i == position:
                return line.strip().split(',')
            
tp = lecture_tuple(DATA, 3240, 822)    
print(tp)

['4894680', '1142830', '886340', '295680', '116071', '28847', '1282', '----------------------------------------------------------------------------------------------------']


# Créer un index

In [3]:
def creation_index_unique(fichier):
    index = {}
    for p, i, t in lecture_sequentielle_par_page(fichier):
        index[t[0]] = (p, i)
        

    # la clé est la valeur du 1er attribut
    # la valeur est un rowid composé de (page, position)

    return index


t1 = time.time()
index1 = creation_index_unique("data.csv")
#print(index1)
print("done in", round(time.time() - t1, 2), "s")

NameError: name 'lecture_sequentielle_par_page' is not defined

In [11]:
def creation_index(fichier,att):
    # creation index qui ne sont pas unique avec table d'hachage
    index = {}
    for p, i, t in lecture_sequentielle_par_page(fichier):
        if t[att] not in index:
            index[t[att]] = []
        index[t[att]].append((p, i))


# Accès par index

## Index unique scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur donnée.

On peut supposer pour simplifier que l'attribut est unique

In [ ]:
def selection_par_index(fichier, valeur_recherchee):
    index = creation_index_unique(fichier)
    p, i = index[str(valeur_recherchee)]
    return lecture_tuple(fichier, p, i)

selection_par_index(DATA,s)


['75417',
 '2048217',
 '714575',
 '538520',
 '208272',
 '75171',
 '59860',
 '----------------------------------------------------------------------------------------------------']

## Index range scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur comprise dans une intervalle donné

In [12]:
def selection_par_index_plage(fichier, borne_inf, borne_sup):
    index = creation_index_unique(fichier)
    for k in index:
        if borne_inf <= int(k) <= borne_sup:
            p, i = index[k]
            print(lecture_tuple(fichier, p, i))


# Mise à jour de données




## Sélectionner un tuple et modifier un de ses attributs



In [ ]:
def selection_tuple_par_index_modif_att(fichier, valeur_recherchee, att_modif, valeur_modif):
    tp = selection_par_index(fichier, valeur_recherchee)
    tp[att_modif] = valeur_modif
    num_page = filtrer_fichier_par_pages(fichier, tp[0])
    with open(page_dir_name(fichier) + f"/page{num_page}", "r+") as fp:
        fp.write(tp)
        fp.seek

## Modifier l'index en conséquence lorsque l'attibut modifié est indexé


# Persistence



## Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement


## Adapter en conséquence les opérations de modification de l'index